In [1]:
import cv2
import os
import glob
import pandas as pd
import requests
import moviepy.editor as mp
import numpy as np


In [2]:
df = pd.read_csv('train.csv')
df.head()


,name,start,end,x,y
0,AvWWVOgaMlk,00:01:30,00:01:33,0.586719,0.311111
1,Y8HMIm8mdns,00:02:51,00:02:54,0.505729,0.240741
2,akwvpAiLFk0,00:02:24,00:02:30,0.698438,0.288889
3,Swss72CHSWg,00:01:30,00:01:37,0.230729,0.204630
4,ymD5uLlLc0g,00:00:36,00:00:40,0.341667,0.475926


In [3]:
#################################################### function to download videos #################################################################################
def download_video(id,start,end,index):
    i=0
    command_yt_dl = f'cmd /c "yt-dlp --external-downloader ffmpeg --external-downloader-args "-ss {start} -to {end}" -f b "https://www.youtube.com/watch?v={id}" -o "data\\videos\\{index}.webm""'
    command_youtube_dl = f'cmd /c "youtube-dl --external-downloader ffmpeg --external-downloader-args "-ss {start} -to {end}" -f best "https://www.youtube.com/watch?v={id}" -o "data\\videos\\{index}.webm""'
    while i<3:
        a= os.system(command_yt_dl)
        print(f'{a}:  {command_yt_dl}')
        if a == 0:
            path = f'data\\videos\\{index}.webm'
            vid = cv2.VideoCapture(path)
            total = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
            print(f'Number Of Frames {index}: {total}')
            if total == -1 or total ==0:
                os.remove(path)
                k =0
                while k<3:
                    a = os.system(command_youtube_dl)
                    print(f'{a}:  {command_youtube_dl}')
                    if a == 0:
                        break
                    k +=1
            else:
                break
        i +=1
        
################################################################ function to check is video is valid or not ######################################################################

def check_video_url(video_id):
    checker_url = "https://www.youtube.com/oembed?url=http://www.youtube.com/watch?v="
    video_url = checker_url + video_id

    request = requests.get(video_url)

    return request.status_code == 200




In [ ]:
##################################################### loop at data and call function download video is valid ###############################################################2
for i in range(0,3):
    id = df['name'][i]
    start = df['start'][i]
    end = df['end'][i]
    if check_video_url(id):
        download_video(id,start,end,i)
    else:
        print(f'{id}: not Found')
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')


In [58]:
##################################################### convert video to audio ###############################################################3333322222222
list_videos = glob.glob('data\\videos\\*')
def convert_video_to_audio(list_videos):
    for  i in range(0, len(list_videos)):
        index = int( list_videos[i].split('\\')[-1].split('.webm')[0])
        clip = mp.VideoFileClip(list_videos[i])
        clip.audio.write_audiofile(f"data\\audio\\{index}.mp3",verbose= False, logger= None)
        
convert_video_to_audio(list_videos)





In [61]:
######################################################################## function that get short distance ################################################## 
def get_short_distance(faces,frame,x_n,y_n,):
    x_frame =int( frame.shape[1] * x_n )
    y_frame =int( frame.shape[0] * y_n )
    distance =[]
    for (x,y,w,h) in faces:
        dis = np.sqrt((x-x_frame)**2 + (y-y_frame)**2)
        distance.append(dis)
    if(len(distance) !=0):
        min_distance = min(distance)
        index = distance.index(min_distance)
        return faces[index]

In [78]:
list_videos = glob.glob('data\\videos\\*')

def extract_person_from_frames(list_videos):
    face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

    for  i in range(0, len(list_videos)):

        index = int( list_videos[i].split('\\')[-1].split('.webm')[0])
        
        vid = cv2.VideoCapture(list_videos[i])
        currentFrame = 0

        mypath = f"data\\frames\\{index}"
        if not os.path.isdir(mypath):
            os.makedirs(mypath)
        try:
            while vid.isOpened():
                success,image = vid.read()
                #print(success)
                #print(currentFrame)
                print(success)
                gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
                faces = face_cascade.detectMultiScale(gray,1.3,5)
                face = get_short_distance(faces,image,df['x'][index],df['y'][index])
                print(face)
            
                try:
                    
                    x,y,w,h = face
                    print(f'{x} = {y}')
                    print(image.shape)
                    face_img = image[y:y+h, x:x+w]
                    resized = cv2.resize(face_img,(128,128))
                    cv2.imwrite(f"data\\frames\\{index}\\{currentFrame}.png",resized)
                    
                except:
                    print('NO face detected')
            
                currentFrame+=1

        except:
            print('Finish')



extract_person_from_frames(list_videos)

True
[558  52 382 382]
558 = 52
(720, 1280, 3)
True
[555  49 384 384]
555 = 49
(720, 1280, 3)
True
[560  51 381 381]
560 = 51
(720, 1280, 3)
True
[559  49 381 381]
559 = 49
(720, 1280, 3)
True
[556  50 379 379]
556 = 50
(720, 1280, 3)
True
[559  51 375 375]
559 = 51
(720, 1280, 3)
True
[561  52 374 374]
561 = 52
(720, 1280, 3)
True
[562  52 377 377]
562 = 52
(720, 1280, 3)
True
[560  54 372 372]
560 = 54
(720, 1280, 3)
True
[556  51 379 379]
556 = 51
(720, 1280, 3)
True
[554  49 381 381]
554 = 49
(720, 1280, 3)
True
[554  49 379 379]
554 = 49
(720, 1280, 3)
True
[552  50 381 381]
552 = 50
(720, 1280, 3)
True
[556  51 379 379]
556 = 51
(720, 1280, 3)
True
[555  52 379 379]
555 = 52
(720, 1280, 3)
True
[554  53 379 379]
554 = 53
(720, 1280, 3)
True
[556  53 379 379]
556 = 53
(720, 1280, 3)
True
[556  50 377 377]
556 = 50
(720, 1280, 3)
True
[554  47 381 381]
554 = 47
(720, 1280, 3)
True
[554  47 380 380]
554 = 47
(720, 1280, 3)
True
[552  48 380 380]
552 = 48
(720, 1280, 3)
True
[556  52